In [1]:
import traceback
import pygame
import carla
import random
import time
import math
import collections
from collections import Counter
import pandas as pd
import itertools
from copy import deepcopy
import warnings
import numpy as np
import random


warnings.filterwarnings("ignore")
from utils import *

pygame 2.5.2 (SDL 2.28.2, Python 3.8.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
distance_to_junc = 20
direction_angle = 20
radius = 100

actor_list = []

for actor in actor_list:
    actor.destroy()
actor_list = []

client = carla.Client("localhost", 2000)
world = client.get_world()

blueprint_library = world.get_blueprint_library()
world_map = world.get_map()
spawn_points = world.get_map().get_spawn_points()
print(world.get_actors())

[Actor(id=152, type=sensor.camera.rgb), Actor(id=151, type=sensor.other.imu), Actor(id=150, type=sensor.other.gnss), Actor(id=149, type=sensor.other.lane_invasion), Actor(id=148, type=sensor.other.collision), Actor(id=147, type=vehicle.dodge.charger_police_2020), Actor(id=146, type=traffic.traffic_light), Actor(id=145, type=traffic.traffic_light), Actor(id=144, type=traffic.traffic_light), Actor(id=143, type=traffic.traffic_light), Actor(id=142, type=traffic.traffic_light), Actor(id=141, type=traffic.traffic_light), Actor(id=140, type=traffic.traffic_light), Actor(id=139, type=traffic.traffic_light), Actor(id=138, type=traffic.traffic_light), Actor(id=137, type=traffic.traffic_light), Actor(id=136, type=traffic.traffic_light), Actor(id=135, type=traffic.traffic_light), Actor(id=134, type=traffic.traffic_light), Actor(id=133, type=traffic.traffic_light), Actor(id=132, type=traffic.traffic_light), Actor(id=131, type=traffic.traffic_light), Actor(id=130, type=traffic.traffic_light), Actor

In [3]:
ego_vehicle = world.get_actor(147)
#second_vehicle = world.get_actor(153)
print(ego_vehicle.get_location())

Location(x=225.252029, y=-364.109375, z=0.000112)


## New Loop 


In [4]:
# initialize sensor data variables
matrix, speed, acceleration, steering_angle, lateral_acceleration, current_yaw, dist_to_lane_center, speed_vehicle_ahead, curvature_degrees_per_meter = None, None, None, None, None, None, None, None, None

previous_speed = 0  # Store the previous speed value in m/s
previous_yaw = 0  # Store the previous yaw angle value
leading_vehicle = None  # Variable to store the leading vehicle reference
previous_time = time.time_ns()

df = initialize_dataframe()
try:
    while True:
        print("----------------------------------")
            
        # update ego location and waypoint
        ego_location = ego_vehicle.get_location()
        ego_waypoint = world_map.get_waypoint(ego_location)
            
        try:
            # create car detection matrix and update parameters
            matrix, params_car_detection = get_car_detection_matrix(**params_car_detection, ego_vehicle=ego_vehicle, ego_waypoint=ego_waypoint, ego_location=ego_location, world=world)
        except Exception as e:
            print("ERROR: In get_car_detection_matrix:")
            print(e)
            print("Traceback:")
            traceback.print_exc()

        current_time = time.time_ns()

        try:
            # get other sensor data
            speed = get_speed(ego_vehicle)
            acceleration = get_acceleration(speed, previous_speed, current_time, previous_time)
            steering_angle = get_steering_angle(ego_vehicle) #TODO angle in degree or radians?
            lateral_acceleration, current_yaw = get_lateral_acceleration(ego_vehicle, speed, previous_yaw, current_time, previous_time)
            dist_to_lane_center = get_dist_to_lane_center(ego_vehicle, world) #in m #TODO needs to be tested with manual car
            speed_vehicle_ahead = get_speed_of_vehicle_ahead(ego_waypoint, world) 

            # Get and print the curvature at the ego vehicle's location in degrees per meter
            curvature_degrees_per_meter = get_curvature_at_location(ego_location, world) 

            # save temporary data
            previous_speed = speed  # Update the previous speed value
            previous_time = current_time
            previous_yaw = current_yaw  # Update the previous yaw angle value
        except Exception as e:
            print("ERROR: In getting sensor data:")
            print(e)
            print("Traceback:")
            print(traceback.print_exc())

        # print sensor data
        print("matrix:")
        if matrix:
            for key, value in matrix.items():
                print(value)
        else:
            print("-- None --")
        print("speed", speed)
        print("acceleration", acceleration)
        print("steering_angle", steering_angle)
        print("lateral_acceleration", lateral_acceleration)
        print("dist_to_lane_center", dist_to_lane_center)
        print("speed_vehicle_ahead", speed_vehicle_ahead)
        print("curvature_degrees_per_meter", curvature_degrees_per_meter)


        # save sensor data in dataframe
        row_data = get_row(matrix)
        row_data["speed"] = speed
        row_data["acceleration"] = acceleration
        row_data["steering_angle"] = steering_angle
        row_data["lateral_acceleration"] = lateral_acceleration
        row_data["dist_to_lane_center"] = dist_to_lane_center
        row_data["speed_vehicle_ahead"] = speed_vehicle_ahead
        row_data["curvature_degrees_per_meter"] = curvature_degrees_per_meter
        df = df._append(row_data, ignore_index=True)

        # clock.tick_busy_loop(60)
        time.sleep(0.3)
        world.tick()
    
finally: 
    display(df)



----------------------------------
Street type: On highway
matrix:
[3, 3, 3, 3, 3, 3, 3, 3]
[3, 3, 3, 3, 3, 3, 3, 3]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0]
[3, 3, 3, 3, 3, 3, 3, 3]
[3, 3, 3, 3, 3, 3, 3, 3]
speed 0.0
acceleration 0.0
steering_angle 0.0
lateral_acceleration -0.0
dist_to_lane_center 0.06472260500758349
speed_vehicle_ahead 0.0
curvature_degrees_per_meter -0.0019389209801150733
----------------------------------
Street type: On highway
matrix:
[3, 3, 3, 3, 3, 3, 3, 3]
[3, 3, 3, 3, 3, 3, 3, 3]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0]
[3, 3, 3, 3, 3, 3, 3, 3]
[3, 3, 3, 3, 3, 3, 3, 3]
speed 0.0
acceleration 0.0
steering_angle 0.0
lateral_acceleration 0.0
dist_to_lane_center 0.06472260500758349
speed_vehicle_ahead 0.0
curvature_degrees_per_meter -0.0019389209801150733
----------------------------------
Street type: On highway
matrix:
[3, 3, 3, 3, 3, 3, 3, 3]
[

,0_0,0_1,0_2,0_3,0_4,0_5,0_6,0_7,1_0,1_1,...,7_5,7_6,7_7,speed,acceleration,steering_angle,lateral_acceleration,dist_to_lane_center,speed_vehicle_ahead,curvature_degrees_per_meter
0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,0.0,0.0,0.0,-0.0,0.064723,0.0,-0.001939
1,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,0.0,0.0,0.0,0.0,0.064723,0.0,-0.001939
2,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,0.0,0.0,0.0,0.0,0.064723,0.0,-0.001939
3,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,0.0,0.0,0.0,0.0,0.064723,0.0,-0.001939
4,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,0.0,0.0,0.0,0.0,0.064723,0.0,-0.001939
5,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,0.0,0.0,0.0,0.0,0.064723,0.0,-0.001939
6,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,0.0,0.0,0.0,0.0,0.064723,0.0,-0.001939
7,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,0.0,0.0,0.0,0.0,0.064723,0.0,-0.001939
8,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,0.0,0.0,0.0,0.0,0.064723,0.0,-0.001939
9,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,0.0,0.0,0.0,0.0,0.064723,0.0,-0.001939


KeyboardInterrupt: 

## Backup

In [5]:
junction = None
same_junction = False
on_junction = False
junctions_detected = False
junction_old = None
wps_old = None
highway_shape_old = None
highway_shape = None
wrong_shape = None
junction_shape = None
matrix = None
junction_id = None
street_type = None 
road_lane_ids = get_all_road_lane_ids(world_map = world.get_map())
on_entry = False
junction_id_skip = None
ego_on_bad_highway_street = False
highway_junction = False
highway_forward_vector = None
df = initialize_dataframe()
while True:
    try:
        print("----------------------------------")
        
        # update ego location and waypoint
        ego_location = ego_vehicle.get_location()
        ego_waypoint = world_map.get_waypoint(ego_location)
        
        # check if ego is on a highway
        ego_on_highway = check_ego_on_highway(ego_location, road_lane_ids, world_map)
        if not ego_on_highway: # check if ego is on the right lane of a highway that is an exit/entry an accounts as another road_id
            left_location = get_location_x_units_to_left(ego_vehicle, 5)
            ego_on_highway = check_ego_on_highway(left_location, road_lane_ids, world_map)
            if ego_on_highway:
                ego_on_bad_highway_street = True # ego_on_bad_highway_street indicates special case, where ego is on a highway but on the right exit/entry lane w another road_id
        else:
            ego_on_bad_highway_street = False
        
        # depending on ego_on_highway, set threshold for junction detection ahead 
        if ego_on_highway:
            distance_to_junc = 80
        else:
            distance_to_junc = 20
        
        # update some boolean variables for junction detection in the city and dynamic highway entry/exit detection, usually ego runs from case 1-5:
        # 5. case: as soon as ego is not on junction anymore, reset variables
        if junctions_detected and not ego_waypoint.is_junction: 
            junctions_detected = False 
            same_junction = False # reset same_junction variable to allow searching for new junction objects ahead 
        # 4. case: ego is on a junction object but junction already fully detected
        elif junctions_detected: 
            pass
        # 2. case: first tick ego is on junction object
        elif ego_waypoint.is_junction and on_junction == False: 
            on_junction = True
        # 3. case: second tick ego is on a junction object
        elif on_junction: 
            # save objects from current junction in xxx_old variables for dynamic highway entry/exit detection in the back of ego
            junction_old = junction
            wps_old = wps
            highway_shape_old = highway_shape
            # junction is now fully detected, reset variables
            on_junction = False
            junctions_detected = True
        # 1. case: as long as ego is approaching a junction 
        elif is_junction_ahead(ego_waypoint, distance_to_junc) and not on_junction: 
            # get junction and check if junction object is on a highway
            junction = get_junction_ahead(ego_waypoint, distance_to_junc)
            junction_id = junction.id
            highway_junction = is_highway_junction(ego_vehicle, ego_waypoint, junction, road_lane_ids, direction_angle, world_map)
            
        # get list with lane_ids of current road of ego
        current_lanes = []
        for id in road_lane_ids:
            if str(ego_waypoint.road_id) == id.split("_")[0]:
                current_lanes.append(int(id.split("_")[1]))
                
        ######################## CASES ########################
        # 1. Case: Ego is on city junction
        # and catch special case gas station junctions: 459, 870, 483, 1249
        # and catch special case traffic light junction on highway: 1368 -> treated as city junction
        if (ego_waypoint.is_junction and not highway_junction and get_junction_ahead(ego_waypoint, distance_to_junc) and ("Town04" in world_map.name and (get_junction_ahead(ego_waypoint, distance_to_junc).id not in [459, 870] or ego_waypoint.get_junction().id in [483, 870, 1249]))) \
            or ("Town04" in world_map.name and ego_waypoint.is_junction and junction and junction.id == 1368): 
            street_type = "On junction"
            print("Street type:", street_type)
            
            # if junction_shape already detected (should be when ego was before junction), detect other cars
            if junction_shape: 
                # detect cars and insert them into matrix
                matrix = detect_surrounding_cars_outside_junction(
                    deepcopy(junction_shape), # deepcopy(junction_shape) to not change original junction_shape for next ticks
                    junction_roads_junction,
                    lanes_all_junction,
                    ego_vehicle,
                    world,
                    world_map,
                    distance_to_junc,
                    junction
                )
                # detect other cars and insert them into matrix
                matrix = detect_cars_inside_junction(
                    matrix, ego_vehicle, wps, yaw, world
                ) 
                # print matrix
                for key, value in matrix.items():
                    print(key, value)
            
            # if junction was not detected properly before, throw error message
            else:
                print("Error in recognizing junction shape")
        
        
        # 2. Case: City junction ahead
        # and catch special case gas station junctions: 459, 870; depending on ego road: [2, 593, 3, 12, 13, 879, 880, 886, 467, 477, 468]
        # and catch special case traffic light junction on highway: 1368 -> treated as city junction
        elif (junction and not highway_junction and get_junction_ahead(ego_waypoint, distance_to_junc) and get_junction_ahead(ego_waypoint, distance_to_junc).id not in [459, 870]) \
            or ("Town04" in world_map.name and ego_waypoint.road_id in [2, 593, 3, 12, 13, 879, 880, 886, 467, 477, 468] and get_junction_ahead(ego_waypoint.next(20)[0], distance_to_junc) and get_junction_ahead(ego_waypoint.next(20)[0], distance_to_junc).id not in [459, 870] and not highway_junction) \
                or ("Town04" in world_map.name and junction and junction.id == 1368 and is_junction_ahead(ego_waypoint, distance_to_junc)):
            street_type = "Junction ahead"
            
            # special case: gas station junctions --> if junction ahead is 459 or 870 update junction bc. the junction objects are too close due to gas station
            if "Town04" in world_map.name and get_junction_ahead(ego_waypoint, distance_to_junc) and get_junction_ahead(ego_waypoint, distance_to_junc).id in [459, 870]:
                junction = get_junction_ahead(ego_waypoint.next(20)[0], distance_to_junc)
            # else, get junction ahead as usual
            else:
                junction = get_junction_ahead(ego_waypoint, distance_to_junc)
            
            wps = junction.get_waypoints(carla.LaneType().Driving)
            
            # special case: traffic light junction on highway logic
            if "Town04" in world_map.name and junction.id == 1368: 
                street_type = "Highway traffic light"
                # ego already on highway, approaching traffic light junction 
                if ego_on_highway: 
                    # remove unnecessary waypoints from junction bc. other lane direction on highway junctions is not relevant
                    starting_wp = get_closest_starting_waypoint(wps, ego_location)
                    wps = remove_wps(starting_wp, wps, world_map)
                    intial_wp = ego_waypoint
                # ego entrying highway traffic light jucntion
                else: 
                    # to build the junction shape correctly, we simulate as if ego is already on highway & approaching the junction 
                    starting_wp, middle_location = get_waypoint_on_highway_junction(junction, ego_waypoint, ego_location, wps, world_map)
                    # remove unnecessary waypoints from junction bc. other lane direction on highway junctions is not relevant
                    wps = remove_wps(starting_wp, wps, world_map)
                    intial_wp = starting_wp.previous(5)[0] # simulated position
            # Every other normal junction
            else: 
                street_type = "Junction ahead"
                intial_wp = ego_waypoint

            # get junction shape and usefull variables, see documentation in utils.py
            junction_shape, lanes_all_junction, junction_roads_junction, yaw, = get_junction_shape(
                ego_vehicle, intial_wp, wps, road_lane_ids, direction_angle, world_map
            )

            if junction_shape != "Error":
                print("Street type:", street_type)
                
                # detect ego and insert it into matrix
                matrix = detect_ego_before_junction(
                    deepcopy(junction_shape), # deepcopy(junction_shape) to not change original junction_shape for next ticks
                    junction_roads_junction,
                    lanes_all_junction,
                    ego_waypoint,
                    distance_to_junc,
                )
                # detect other cars and insert them into matrix
                matrix = detect_surrounding_cars_outside_junction(
                    matrix,
                    junction_roads_junction,
                    lanes_all_junction,
                    ego_vehicle,
                    world,
                    world_map,
                    distance_to_junc,
                    junction
                )
                # detect other cars and insert them into matrix
                matrix = detect_cars_inside_junction(
                    matrix, ego_vehicle, wps, yaw, world
                )
                # print matrix
                for key, value in dict(matrix).items():
                    print(key, value)
            
            # if junction was not detected properly before, throw error message
            else:
                print("Error in recognizing junction shape")


        # 3. Highway entry/exit in front or behind
        elif (is_junction_ahead(ego_waypoint, 80) or (is_junction_behind(ego_waypoint, 40) and street_type == "Highway with entry/exit")) and not on_entry and highway_junction and ego_on_highway and junction_id_skip != junction.id and not check_ego_exit_highway(ego_vehicle, ego_waypoint, highway_forward_vector, highway_shape, current_lanes):
            street_type = "Highway with entry/exit"
            print("Street type:", street_type)
            
            # Save ego forward vector for checking if ego is exiting highway: exiting if deviation from forward vector too high
            if distance(ego_location, junction.bounding_box.location) < 30 and not forward_vector_set:
                highway_forward_vector = ego_vehicle.get_transform().get_forward_vector()
                highway_forward_location = ego_vehicle.get_location()
                forward_vector_set = True
                exit_over = False
            
            # if new junction ahead
            if not same_junction and is_junction_ahead(ego_waypoint, 80):
                forward_vector_set = False # reset forward vector
                same_junction = True # set same_junction to true to not detect same junction again 

                # get raw junction waypoints and prepare them to determine highway junction shape
                junction = get_junction_ahead(ego_waypoint, 80)
                wps = junction.get_waypoints(carla.LaneType().Driving)
                closest_waypoint = get_closest_starting_waypoint(wps, ego_location)
                wps = prepare_waypoints(closest_waypoint, wps, world_map)
                highway_shape = get_highway_shape(wps, junction, ego_waypoint, world_map)
                
                # print highway junction shape and number of straight highway lanes
                if not highway_shape is None:
                    print(highway_shape[0], highway_shape[1])
            
            # in case a junction object is ahead but the object is not relevant (e.g. on other lane direction) or is not detected properly, then use backuped junction object if available
            if (highway_shape is None) and (not junction_old is None):
                junction = junction_old
                wps = wps_old
                highway_shape = highway_shape_old
                wrong_shape = True
            
            # create basic matrix of highway scenario without entry/exit detection  
            if ego_on_bad_highway_street: # ego_on_bad_highway_street indicates that ego is on the right lane of a highway that is an exit/entry and accounts as another road_id
                matrix = create_basic_matrix(left_location, road_lane_ids, world_map, True, ego_on_bad_highway_street) #  pass "left_location" to create matrix based on highway road_id not entry/exit road_id
            else:
                matrix = create_basic_matrix(ego_location, road_lane_ids, world_map, False, ego_on_bad_highway_street)                
            
            if matrix:
                # detect other cars and insert them into matrix
                matrix, cars_on_entryExit = detect_surronding_cars(
                    ego_location, ego_vehicle, matrix, road_lane_ids, world, radius, ego_on_highway, highway_shape
                )
                # update matrix with highway entry/exit detection
                if not highway_shape is None and (highway_shape[0] != "normal_highway" or wrong_shape):
                    matrix = update_matrix(world_map, ego_vehicle, ego_location, highway_shape, wps, matrix, junction, cars_on_entryExit, direction_angle) 
                    if not junction_old is None and junction_old.id != junction.id:
                        # update matrix with highway entry/exit detection for junction behind: dynamically move entry/exit out of matrix behind ego
                        matrix = update_matrix(world_map, ego_vehicle, ego_location, highway_shape_old, wps_old, matrix, junction_old, cars_on_entryExit, direction_angle) 
            
            # print matrix   
            for key, value in matrix.items():
                print(value)
            print("----------------------------------------------------------------------------------------------------------------------")

        # 4. ego exiting Highway
        elif check_ego_exit_highway(ego_vehicle, ego_waypoint, highway_forward_vector, highway_shape, current_lanes) and not exit_over:
            street_type = "On highway exit"
            print("Street type:", street_type)
            
            if highway_forward_vector and distance(highway_forward_location, ego_location) > 60:
                highway_forward_vector = None
                highway_forward_location = None
            
            junction = ego_waypoint.get_junction()
            if not junction:
                exit_over = True
                continue
            junction_id_skip = junction.id

            wps = junction.get_waypoints(carla.LaneType().Driving)
            closest_waypoint, middle_location = get_waypoint_on_highway_junction(junction, ego_waypoint, ego_location, wps, world_map)
            wps = prepare_waypoints(closest_waypoint, wps, world_map)
            highway_shape = get_highway_shape(wps, junction, ego_waypoint, world_map)

            on_entry = True
            if (highway_shape is None) and (not junction_old is None):
                same_junction = False
                junction = junction_old
                wps = wps_old
                highway_shape = highway_shape_old


            matrix = create_basic_matrix(middle_location, road_lane_ids, world_map, ghost=True)
            if matrix:
                matrix, cars_on_entryExit = detect_surronding_cars(
                    middle_location, ego_vehicle, matrix, road_lane_ids, world, radius, True, highway_shape, ghost=True
                )

                if not highway_shape is None:
                    matrix  = update_matrix(world_map, ego_vehicle, middle_location, highway_shape, wps, matrix, junction, cars_on_entryExit, direction_angle, ghost=True, on_entry=False) 
                    # if not junction_old is None and junction_old.id != junction.id:
                    #     matrix = update_matrix(world_map, ego_vehicle, middle_location, highway_shape_old, wps_old, matrix, closest_waypoint, junction_old, cars_on_entryExit, True, on_entry) 
                    
            for key, value in matrix.items():
                print(value)
            print("----------")


        # 5. Highway entry
        elif (is_junction_ahead(ego_waypoint, 20) or (len(current_lanes) <= 1 and ego_waypoint.is_junction)) and highway_junction and street_type in  ["Non highway street", "On junction", "On highway entry", "On highway exit"]:

            street_type = "On highway entry"
            print("Street type:", street_type)
            on_entry = True
            
            junction = get_junction_ahead(ego_waypoint, 20)
            if not junction:
                junction = ego_waypoint.get_junction()
            junction_id_skip = junction.id

            wps = junction.get_waypoints(carla.LaneType().Driving)
            closest_waypoint, middle_location = get_waypoint_on_highway_junction(junction, ego_waypoint, ego_location, wps, world_map)
            wps = prepare_waypoints(closest_waypoint, wps, world_map)
            highway_shape = get_highway_shape(wps, junction, ego_waypoint, world_map)

            on_entry = True
            if (highway_shape is None) and (not junction_old is None):
                same_junction = False
                junction = junction_old
                wps = wps_old
                highway_shape = highway_shape_old


            matrix = create_basic_matrix(middle_location, road_lane_ids, world_map, ghost=True)
            if matrix:
                matrix, cars_on_entryExit = detect_surronding_cars(
                    middle_location, ego_vehicle, matrix, road_lane_ids, world, radius, True, highway_shape, ghost=True
                )

                if not highway_shape is None:
                    matrix = update_matrix(world_map, ego_vehicle, middle_location, highway_shape, wps, matrix, junction, cars_on_entryExit, direction_angle, ghost=True, on_entry=True) 

            for key, value in matrix.items():
                print(value)
            print("----------")

        # 6. Normal Road
        else:
            highway_forward_vector = None
            highway_forward_location = None
            on_junction = False
            junction_old = None
            on_entry = False
            same_junction = False
            if ego_on_highway:
                street_type = "On highway"
            else:
                street_type = "Non highway street"
            print("Street type:", street_type)
            matrix = create_basic_matrix(ego_location, road_lane_ids, world_map)
            # special road (gas station)
            if "Town04" in world_map.name and ego_waypoint.road_id in  [467, 477]:
                matrix = collections.OrderedDict([("468_-1", [0, 0, 0, 0, 0, 0, 0, 0]) if k == 'No_opposing_direction' else (k, v) for k, v in matrix.items()])
            elif "Town04" in world_map.name and ego_waypoint.road_id == 468:
                matrix = collections.OrderedDict([("467_1", [0, 0, 0, 0, 0, 0, 0, 0]) if k == 'No_opposing_direction' else (k, v) for k, v in matrix.items()])        
            
            if matrix:
                matrix, _ = detect_surronding_cars(
                    ego_location, ego_vehicle, matrix, road_lane_ids, world, radius, ego_on_highway, highway_shape
                )
                for key, value in matrix.items():
                    print(value)


        df = safe_data(ego_vehicle, matrix, street_type, df)

        # clock.tick_busy_loop(60)
        time.sleep(0.3)
        world.tick()
    
    except Exception as e:
        print(e)
        print("ERROR")
        continue




time.sleep(0.5)


----------------------------------
Street type: Non highway street
[3, 3, 3, 3, 3, 3, 3, 3]
[3, 3, 3, 3, 3, 3, 3, 3]
[3, 3, 3, 3, 3, 3, 3, 3]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0]
[3, 3, 3, 3, 3, 3, 3, 3]
[3, 3, 3, 3, 3, 3, 3, 3]
[3, 3, 3, 3, 3, 3, 3, 3]
----------------------------------
Street type: On highway entry
[3, 3, 3, 3, 3, 3, 3, 3]
[3, 3, 3, 3, 3, 3, 3, 3]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[3, 0, 3, 1, 3, 3, 3, 3]
[3, 0, 3, 0, 3, 3, 3, 3]
----------
----------------------------------
Street type: On highway entry
[3, 3, 3, 3, 3, 3, 3, 3]
[3, 3, 3, 3, 3, 3, 3, 3]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[3, 0, 3, 1, 3, 3, 3, 3]
[3, 0, 3, 0, 3, 3, 3, 3]
----------
----------------------------------
Street type: On highway entry
[3, 3, 3, 3, 3, 3, 3, 3]
[3, 3, 3, 3, 3, 3, 3, 3]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 

KeyboardInterrupt: 

In [ ]:
df.to_csv("test.csv")